# Workshop notebook

This Python+Jupyter notebook has example calculations useful for anyone who attended my ISAHP2020 workshop on ANP Row Sensitivity, and anyone else interested in the topic.

My slideshow is available [on google slide](https://docs.google.com/presentation/d/1AK6TGB_45D-98Un3vMhOV-Fq5G4gftQbqbj1lMFMLtc/edit?usp=sharing)

## Initial imports

In [1]:
from pyanp import limitmatrix as lm
from pyanp import anp as anp
from pyanp import rowsens as rowsens
import numpy as np
import pandas as pd
from IPython.display import Math, HTML
#Import my custom functions also
from isahp2020 import *
# Let's only display 4 decimal places
np.set_printoptions(precision=4)

## Setup our model

In [2]:
# The model consists of a model with 3 criteria, 2 alts, fully connected.
# We give it in terms of the unscaled super matrix and the cluster matrix

#What is the index of the alternatives, this is used in a lot of my isahp2020 function
alts = [3,4]
unscaled = np.array([
    [0.60, 0.20, 0.40, 0.20, 0.10],
    [0.30, 0.20, 0.35, 0.30, 0.30],
    [0.10, 0.60, 0.25, 0.50, 0.60],
    [0.20, 0.90, 0.40, 0.30, 0.25],
    [0.80, 0.10, 0.60, 0.70, 0.75]
])
cluster = np.array([
    [0.8, 0.6],
    [0.2, 0.4]
])
scaled = scale_mat(unscaled, cluster, [3,2])
limit = lm.calculus(scaled)
alt_scores = lmsynth(scaled, alts)
display(HTML("<h3>The supermatrix is:</h3>"))
display(scaled)
display(HTML("<h3>The limit matrix is:</h3>"))
display(limit)
display(HTML("<h3>The resulting priority vector is:</h3>"))
display(alt_scores)

array([[0.48, 0.16, 0.32, 0.12, 0.06],
       [0.24, 0.16, 0.28, 0.18, 0.18],
       [0.08, 0.48, 0.2 , 0.3 , 0.36],
       [0.04, 0.18, 0.08, 0.12, 0.1 ],
       [0.16, 0.02, 0.12, 0.28, 0.3 ]])

array([[0.269 , 0.269 , 0.269 , 0.269 , 0.269 ],
       [0.2181, 0.2181, 0.2181, 0.2181, 0.2181],
       [0.2629, 0.2629, 0.2629, 0.2629, 0.2629],
       [0.098 , 0.098 , 0.098 , 0.098 , 0.098 ],
       [0.152 , 0.152 , 0.152 , 0.152 , 0.152 ]])

array([0.392, 0.608])

# Hand calculations of row sensitivity

In this section we perform both calculations "by hand"

## Scale down, p=0.25 and p_0=0.5

### A helper function

In [3]:
def sens_rescale_column(original_mat, new_mat, sensitivity_row, column):
    # Now column minus the entry in row 1
    oldSum = sum(original_mat[:,column])-original_mat[sensitivity_row,column]
    # And column 0 minus the new entry must add to 1-new_row_1_entry
    newSum = 1-new_mat[sensitivity_row,column]
    # Now we scale each entry except 0 by newSum/oldSum
    for row in range(len(original_mat)):
        if row != 1:
            new_mat[row,column]=newSum/oldSum * original_mat[row,column]
    #Let's see what we look now
    display(HTML("<h3>After column "+str(column+1)+" scaled</h3>"))
    display(new_mat)

### Now do the calculation:

1. Scale down the row
2. Rescale the rest of column 1
2. Rescale the rest of column 2
2. Rescale the rest of column 3
2. Rescale the rest of column 4
2. Rescale the rest of column 5


In [4]:
p=0.25
p0=0.5
# Created matrix to store the result in
scaledRow1Down025 = np.array(scaled)
# First we scale down our row by p/p0 = 0.25/0.5 = 0.5
scaledRow1Down025[1,:] = p/p0 * scaled[1,:]
# Now let's see the result
display(HTML("<h3>Original matrix</h3>"))
display(scaled)
display(HTML("<h3>Row 1 scaled</h3>"))
display(scaledRow1Down025)

array([[0.48, 0.16, 0.32, 0.12, 0.06],
       [0.24, 0.16, 0.28, 0.18, 0.18],
       [0.08, 0.48, 0.2 , 0.3 , 0.36],
       [0.04, 0.18, 0.08, 0.12, 0.1 ],
       [0.16, 0.02, 0.12, 0.28, 0.3 ]])

array([[0.48, 0.16, 0.32, 0.12, 0.06],
       [0.12, 0.08, 0.14, 0.09, 0.09],
       [0.08, 0.48, 0.2 , 0.3 , 0.36],
       [0.04, 0.18, 0.08, 0.12, 0.1 ],
       [0.16, 0.02, 0.12, 0.28, 0.3 ]])

In [5]:
# Above we created a simple function to do the column
# rescaling and print out the intermediate step.
# Let's just use that in a for loop
for column in [0, 1, 2, 3, 4]:
    sens_rescale_column(scaled, scaledRow1Down025, 1, column)

array([[0.5558, 0.16  , 0.32  , 0.12  , 0.06  ],
       [0.12  , 0.08  , 0.14  , 0.09  , 0.09  ],
       [0.0926, 0.48  , 0.2   , 0.3   , 0.36  ],
       [0.0463, 0.18  , 0.08  , 0.12  , 0.1   ],
       [0.1853, 0.02  , 0.12  , 0.28  , 0.3   ]])

array([[0.5558, 0.1752, 0.32  , 0.12  , 0.06  ],
       [0.12  , 0.08  , 0.14  , 0.09  , 0.09  ],
       [0.0926, 0.5257, 0.2   , 0.3   , 0.36  ],
       [0.0463, 0.1971, 0.08  , 0.12  , 0.1   ],
       [0.1853, 0.0219, 0.12  , 0.28  , 0.3   ]])

array([[0.5558, 0.1752, 0.3822, 0.12  , 0.06  ],
       [0.12  , 0.08  , 0.14  , 0.09  , 0.09  ],
       [0.0926, 0.5257, 0.2389, 0.3   , 0.36  ],
       [0.0463, 0.1971, 0.0956, 0.12  , 0.1   ],
       [0.1853, 0.0219, 0.1433, 0.28  , 0.3   ]])

array([[0.5558, 0.1752, 0.3822, 0.1332, 0.06  ],
       [0.12  , 0.08  , 0.14  , 0.09  , 0.09  ],
       [0.0926, 0.5257, 0.2389, 0.3329, 0.36  ],
       [0.0463, 0.1971, 0.0956, 0.1332, 0.1   ],
       [0.1853, 0.0219, 0.1433, 0.3107, 0.3   ]])

array([[0.5558, 0.1752, 0.3822, 0.1332, 0.0666],
       [0.12  , 0.08  , 0.14  , 0.09  , 0.09  ],
       [0.0926, 0.5257, 0.2389, 0.3329, 0.3995],
       [0.0463, 0.1971, 0.0956, 0.1332, 0.111 ],
       [0.1853, 0.0219, 0.1433, 0.3107, 0.3329]])

In [6]:
# Let's look at the rowsens calculation to perform this directly, it should be the same as the above
display(HTML("<h3>Using the simple rowsens.row_adjust() function instead "))
display(rowsens.row_adjust(scaled, 1, 0.25, p0mode=0.5))

array([[0.5558, 0.1752, 0.3822, 0.1332, 0.0666],
       [0.12  , 0.08  , 0.14  , 0.09  , 0.09  ],
       [0.0926, 0.5257, 0.2389, 0.3329, 0.3995],
       [0.0463, 0.1971, 0.0956, 0.1332, 0.111 ],
       [0.1853, 0.0219, 0.1433, 0.3107, 0.3329]])

In [7]:
# Let's get our new priority
new_alt_scores = lmsynth(scaledRow1Down025, alts)
display(HTML("<h3>Original alt scores were</h3>"))
display(alt_scores)
display(HTML("<h3>New alternative scores after scaling down are</h3>"))
display(new_alt_scores)
display(HTML("Notice, this is a relatively small change, which is common for scaling downward"))

array([0.392, 0.608])

array([0.3305, 0.6695])

## Let's scale upward p=0.75 and p0=0.5

In [8]:
p=0.75
p0=0.5
# Allocate for our result
scaledRow1Up075 = np.array(scaled)
# First we need to get the second row
orig_row = scaled[1,:]
one_minus_orig_row = 1 - orig_row
# Now we scale down 1-orig_row by (1-p)/(1-p0)
new_one_minus = (1-p)/(1-p0) * one_minus_orig_row
# Our new row is one minus the above
scaledRow1Up075[1,:] = 1 - new_one_minus
# Let's verify things look correct
display(HTML("<h3>After scaling up row 1</h3>"))
display(scaledRow1Up075)

array([[0.48, 0.16, 0.32, 0.12, 0.06],
       [0.62, 0.58, 0.64, 0.59, 0.59],
       [0.08, 0.48, 0.2 , 0.3 , 0.36],
       [0.04, 0.18, 0.08, 0.12, 0.1 ],
       [0.16, 0.02, 0.12, 0.28, 0.3 ]])

In [9]:
# Now we scale each column like we did before
# and we already have functions to do this :)
# Let's wrap this in a for loop to make it quick
for column in [0, 1, 2, 3, 4]:
    sens_rescale_column(scaled, scaledRow1Up075, 1, column)

array([[0.24, 0.16, 0.32, 0.12, 0.06],
       [0.62, 0.58, 0.64, 0.59, 0.59],
       [0.04, 0.48, 0.2 , 0.3 , 0.36],
       [0.02, 0.18, 0.08, 0.12, 0.1 ],
       [0.08, 0.02, 0.12, 0.28, 0.3 ]])

array([[0.24, 0.08, 0.32, 0.12, 0.06],
       [0.62, 0.58, 0.64, 0.59, 0.59],
       [0.04, 0.24, 0.2 , 0.3 , 0.36],
       [0.02, 0.09, 0.08, 0.12, 0.1 ],
       [0.08, 0.01, 0.12, 0.28, 0.3 ]])

array([[0.24, 0.08, 0.16, 0.12, 0.06],
       [0.62, 0.58, 0.64, 0.59, 0.59],
       [0.04, 0.24, 0.1 , 0.3 , 0.36],
       [0.02, 0.09, 0.04, 0.12, 0.1 ],
       [0.08, 0.01, 0.06, 0.28, 0.3 ]])

array([[0.24, 0.08, 0.16, 0.06, 0.06],
       [0.62, 0.58, 0.64, 0.59, 0.59],
       [0.04, 0.24, 0.1 , 0.15, 0.36],
       [0.02, 0.09, 0.04, 0.06, 0.1 ],
       [0.08, 0.01, 0.06, 0.14, 0.3 ]])

array([[0.24, 0.08, 0.16, 0.06, 0.03],
       [0.62, 0.58, 0.64, 0.59, 0.59],
       [0.04, 0.24, 0.1 , 0.15, 0.18],
       [0.02, 0.09, 0.04, 0.06, 0.05],
       [0.08, 0.01, 0.06, 0.14, 0.15]])

In [10]:
# Let's look at the rowsens calculation to perform this directly, it should be the same as the above
display(HTML("<h3>Using the simple rowsens.row_adjust() function instead "))
display(rowsens.row_adjust(scaled, 1, 0.75, p0mode=0.5))

array([[0.24, 0.08, 0.16, 0.06, 0.03],
       [0.62, 0.58, 0.64, 0.59, 0.59],
       [0.04, 0.24, 0.1 , 0.15, 0.18],
       [0.02, 0.09, 0.04, 0.06, 0.05],
       [0.08, 0.01, 0.06, 0.14, 0.15]])

In [11]:
# Let's get our new priority
new_alt_scores = lmsynth(scaledRow1Up075, alts)
display(HTML("<h3>Original alt scores were</h3>"))
display(alt_scores)
display(HTML("<h3>New alternative scores after scaling up are</h3>"))
display(new_alt_scores)
display(HTML("Notice, this is a <b>big change</b> which happens more frequently when scaling upward"))

array([0.392, 0.608])

array([0.6251, 0.3749])

# Sensitivity calculations

## Influence

* Which of the 3 criteria influences the node scores the most?

In [12]:
display(HTML("The first column is the original score, the 2nd and 3rd are the difference between the original score and the adjusted priority"))
for node in [0, 1, 2]:
    infl_050 = influence_priority(scaled, node, 0.5, alts, p0mode=0.5)
    infl_075 = influence_priority(scaled, node, 0.75, alts, p0mode=0.5)
    diff075=infl_075-infl_050
    infl_090 = influence_priority(scaled, node, 0.90, alts, p0mode=0.5)
    diff090=infl_090-infl_050
    display(HTML("<h3>Influence scores for node "+str(node)+"</h3>"))
    df = pd.DataFrame({"0.50":infl_050, "0.75":diff075, "0.90":diff090})
    new_index = len(df)
    df.loc[len(df)] = [0, sum(np.abs(diff075)), sum(np.abs(diff090))]
    df.rename(index={new_index:"Total"}, inplace=True)
    display(df)


,0.50,0.75,0.90
0,0.392023,-0.092555,-0.151202
1,0.607977,0.092555,0.151202
Total,0.000000,0.185110,0.302404


,0.50,0.75,0.90
0,0.392023,0.233096,0.392795
1,0.607977,-0.233096,-0.392795
Total,0.000000,0.466192,0.785589


,0.50,0.75,0.90
0,0.392023,0.011019,0.01125
1,0.607977,-0.011019,-0.01125
Total,0.000000,0.022038,0.02250


### Influence conclusions

* Clearly node 1 has the most influence, having a total change of alternative score of 0.446192 and p=0.75
* And then 0.785589 at p=0.90